In [3]:
import pandas as pd
import numpy as np

In [24]:
data = pd.read_csv('Banco de Súmulas - Sumulas.csv')

# choose class
data = data[data['Classe Vela'] == 'IQFOIL Fem.']

In [6]:
# create dictionary to map competitor names to unique indices
competitor_to_index = {competitor: idx for idx, competitor in enumerate(data['Nome Competidor'].unique())}

# create matrix of size (n_players, n_players) with zeros
n_players = len(competitor_to_index)
matrix_alpha = np.zeros((n_players, n_players))
matrix_beta = np.zeros((n_players, n_players))

# iterate through competitions
for competition in data['Nome Competição'].unique():
    # get data for this competition
    data_competition = data[data['Nome Competição'] == competition]
    data_competition = data_competition.drop_duplicates(subset='Nome Competidor', keep='first')

    # get list of competitors in this competition
    competitors = data_competition['Nome Competidor'].unique()
    # get number of competitors in this competition
    n_players = len(competitors)
    
    # iterate through competitors in this competition setting (i, j) = 1 if i beats j in this competition
    # get i and j from the dictionary
    for i in range(n_players):
        for j in range(n_players):
            competitor_i = competitors[i]
            competitor_j = competitors[j]
            posicao_i = data_competition[(data_competition['Nome Competidor'] == competitor_i)]['Posição Geral'].values[0]
            posicao_j = data_competition[(data_competition['Nome Competidor'] == competitor_j)]['Posição Geral'].values[0]

            # get index of competitor i and j in the matrix
            index_i = competitor_to_index[competitor_i]
            index_j = competitor_to_index[competitor_j]

            if posicao_i < posicao_j:
                # update the cumulative matrix
                matrix_alpha[index_i][index_j] += 1
                # update matrix beta with difference in points
                matrix_beta[index_i][index_j] += (posicao_j - posicao_i)/(posicao_i + posicao_j)

In [22]:
# checking results
# write_matrix = pd.DataFrame(matrix_alpha)

# # columns and index are the names of the competitors
# write_matrix.columns = competitor_to_index.keys()
# write_matrix.index = competitor_to_index.keys()

# write_matrix

In [18]:
# Keener's ranking

# i dont think we consider at all the fact that some competitors may have been in more competitions than others
# so we should normalize the matrix by the number of competitions each competitor was in
# DO IT LATER

# W = matrix_alpha
W = matrix_alpha

# get d vector, d=(W + W^T)1, where 1 is a vector of ones 
d = np.dot(W + W.T, np.ones(W.shape[0]))

# get D matrix, D = diag(d)
D = np.diag(d)

# perron frobeniun eigenvector of D^-1 W
eigenvalues, eigenvectors = np.linalg.eig(np.dot(np.linalg.inv(D), W))
idx = np.argmax(eigenvalues)
eigenvector = eigenvectors[idx]

# get the ranking
ranking = pd.DataFrame(eigenvector, index=competitor_to_index.keys(), columns=['ranking'])
ranking = ranking.sort_values(by='ranking', ascending=False)

ranking.head()

,ranking
Emma WILSON,0.234902+0.000000j
Imogen SILLS,0.203898+0.000000j
Aimee BRIGHT,0.203898-0.000000j
Helene NOESMOEN,0.196717+0.000000j
Nicole van der VELDEN,0.193745+0.000000j


In [23]:
# keener's ranking with beta matrix

# i dont think we consider at all the fact that some competitors may have been in more competitions than others
# so we should normalize the matrix by the number of competitions each competitor was in
# DO IT LATER

# S = matrix_beta
S = matrix_beta

# get d vector, d=(S + S^T)1, where 1 is a vector of ones
d = np.dot(S + S.T, np.ones(S.shape[0]))

# get D matrix, D = diag(d)
D = np.diag(d)

# create empty K matrix with same size as matrix_beta
K = np.zeros(matrix_beta.shape)

# for entry (i, j) in beta matrix, define h = ((i, j) + 1)/((i, j) + (j, i) + 2)
for i in range(n_players):
    for j in range(n_players):
        x = (matrix_beta[i][j] + 1)/(matrix_beta[i][j] + matrix_beta[j][i] + 2)
        h = 1/2 + 1/2 * np.sign(x - 1/2) * np.sqrt(abs(2*x - 1))
        K[i][j] = h

# perron frobeniun eigenvector of D^-1 K
eigenvalues, eigenvectors = np.linalg.eig(np.dot(np.linalg.inv(D), K))
idx = np.argmax(eigenvalues)
eigenvector = eigenvectors[idx]

# get the ranking
ranking = pd.DataFrame(eigenvector, index=competitor_to_index.keys(), columns=['ranking'])
ranking = ranking.sort_values(by='ranking', ascending=False)

ranking.head()

,ranking
Onishi FUJIKO,0.249506-0.000000j
Helena WANSER,0.249506+0.000000j
Katerina SVIKOVA,0.194016+0.081216j
Marion COUTURIER,0.194016-0.081216j
Saskia SILLS,0.178220+0.019858j
